In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


Chamando as bases

In [3]:
df18 = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name='Educação', usecols=['Cod Uf','EEDU21'])
df18

,Cod Uf,EEDU21
0,11,Sim
1,12,Sim
2,13,Sim
3,14,Não
4,15,Não
5,16,Sim
6,17,Sim
7,21,Sim
8,22,Sim
9,23,Não


In [4]:
df21 = pd.read_excel('ESTADIC//Base_ESTADIC_2021.xlsx', sheet_name='Educação', usecols=['Cod UF','EEDU21'])
df21

,Cod UF,EEDU21
0,11,Não
1,12,Sim
2,13,Sim
3,14,Sim
4,15,Não
5,16,Sim
6,17,Sim
7,21,Sim
8,22,Sim
9,23,Não


In [5]:
uf = pd.read_excel('ESTADIC//Base_ESTADIC_2018.xlsx', sheet_name = 'Variáveis externas', usecols=[1,2,3])
uf

,COD UF,UF,NOME UF
0,11,RO,RONDÔNIA
1,12,AC,ACRE
2,13,AM,AMAZONAS
3,14,RR,RORAIMA
4,15,PA,PARÁ
5,16,AP,AMAPÁ
6,17,TO,TOCANTINS
7,21,MA,MARANHÃO
8,22,PI,PIAUÍ
9,23,CE,CEARÁ


Renomeando as colunas

In [6]:
df18= df18.rename(columns={'Cod Uf':'cod_uf',
                        'EEDU21':'plano_carreira_vigente'}) 

In [7]:
df21= df21.rename(columns={'Cod UF':'cod_uf',
                           'EEDU21':'plano_carreira_vigente'}) 

In [8]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [9]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [10]:
x= uf.pivot_table(columns=('COD UF', 'UF', 'NOME UF'), aggfunc='size')


In [11]:
uf = pd.DataFrame(x).reset_index()[['COD UF', 'UF', 'NOME UF']]

In [12]:
df = df.merge(uf, right_on='COD UF',left_on='cod_uf') #adicionando sigla e nome das UFs


In [13]:
df = df.drop(['COD UF'], axis=1) #eliminando coluna repetida

In [14]:
df= df.rename(columns={'UF':'sigla_uf',
                'NOME UF':'uf'})

In [15]:
df['uf'] = df['uf'].str.title()

In [16]:
df = df[['ano','cod_uf','sigla_uf','uf','plano_carreira_vigente']]

In [19]:
df['ano'].unique()

array([2018, 2021], dtype=int64)

In [17]:
df

,ano,cod_uf,sigla_uf,uf,plano_carreira_vigente
0,2018,11,RO,Rondônia,Sim
1,2021,11,RO,Rondônia,Não
2,2018,12,AC,Acre,Sim
3,2021,12,AC,Acre,Sim
4,2018,13,AM,Amazonas,Sim
5,2021,13,AM,Amazonas,Sim
6,2018,14,RR,Roraima,Não
7,2021,14,RR,Roraima,Sim
8,2018,15,PA,Pará,Não
9,2021,15,PA,Pará,Não


Subindo para o GBQ

In [21]:
client = bigquery.Client()
dataset_ref = client.dataset('estrutura_organizacional_carreiras')

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ano                     54 non-null     int64 
 1   cod_uf                  54 non-null     int64 
 2   sigla_uf                54 non-null     object
 3   uf                      54 non-null     object
 4   plano_carreira_vigente  54 non-null     object
dtypes: int64(2), object(3)
memory usage: 2.5+ KB


In [23]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano da apuração daquele dado'),
        bigquery.SchemaField('cod_uf','INTEGER',description='Código do IBGE da UF'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da UF'),
        bigquery.SchemaField('uf','STRING',description='Nome da UF'),        
        bigquery.SchemaField('plano_carreira_vigente','STRING',description='Se existe plano de carreira vigente para os profissionais da educação não docentes')]


In [24]:
table_ref = dataset_ref.table('ESTADIC_existencia_plano_carreira_exceto_docentes')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=fe5712a3-d21c-46e5-a39b-6677643e20f2>